## Introduction

This notebook is to clean and prepare Education data for analysis, ultimately to determine if school performance is predicted by socioeconomic factors

## Import libraries

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno

# Train-test splits
from sklearn.model_selection import train_test_split

# Model preprocessing
from sklearn.preprocessing import StandardScaler

# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

## The data

This project utilizes two data sets. The primary data set is the EdGap data set from [EdGap.org](https://www.edgap.org/#5/37.875/-96.987). This data set from 2016 includes information about average ACT or SAT scores for schools and several socioeconomic characteristics of the school district. The secondary data set is basic information about each school from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp).





### EdGap data

All socioeconomic data (household income, unemployment, adult educational attainment, and family structure) are from the Census Bureau's American Community Survey.

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) report that ACT and SAT score data is from each state's department of education or some other public data release. The nature of the other public data release is not known.

The quality of the census data and the department of education data can be assumed to be reasonably high.

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) do not indicate that they processed the data in any way. The data were assembled by the [EdGap.org](https://www.edgap.org/#5/37.875/-96.987) team, so there is always the possibility for human error. Given the public nature of the data, we would be able to consult the original data sources to check the quality of the data if we had any questions.

### School information data

The school information data is from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp). This data set consists of basic identifying information about schools and can be assumed to be of reasonably high quality. As for the EdGap.org data, the school information data is public, so we would be able to consult the original data sources to check the quality of the data if we had any questions.


## Load the data

Load the EdGap
 data set

In [35]:
edgap = pd.read_excel('https://github.com/galenegan/DATA-3320/raw/main/education/EdGap_data.xlsx')

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Load the school information data

In [36]:
!wget -O school_info.csv https://drive.usercontent.google.com/u/0/uc?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP&export=download

--2024-04-23 02:59:44--  https://drive.usercontent.google.com/u/0/uc?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.202.132, 2607:f8b0:4001:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.202.132|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.usercontent.google.com/uc?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP [following]
--2024-04-23 02:59:44--  https://drive.usercontent.google.com/uc?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP
Reusing existing connection to drive.usercontent.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP [following]
--2024-04-23 02:59:44--  https://drive.usercontent.google.com/download?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP
Reusing existing connection to drive.usercontent.google.com:443.
HTTP request sent, awaitin

In [37]:
school_info = pd.read_csv('school_info.csv', encoding= 'unicode_escape')

<ipython-input-37-9aca6f144c98>:1: DtypeWarning: Columns (6,9,14,15,18,19,21,22,25,26,29,31,35,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  school_info = pd.read_csv('school_info.csv', encoding= 'unicode_escape')


## Explore the contents of the data sets

#### Check the first few columns of each data frame

In [38]:
edgap.head()

,NCESSCH School ID,CT Unemployment Rate,CT Pct Adults with College Degree,CT Pct Childre In Married Couple Family,CT Median Household Income,School ACT average (or equivalent if SAT score),School Pct Free and Reduced Lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [39]:
school_info.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,...,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2016-2017,1,ALABAMA,AL,Sequoyah Sch - Chalkville Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
1,2016-2017,1,ALABAMA,AL,Camps,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
2,2016-2017,1,ALABAMA,AL,Det Ctr,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
3,2016-2017,1,ALABAMA,AL,Wallace Sch - Mt Meigs Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
4,2016-2017,1,ALABAMA,AL,McNeel Sch - Vacca Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported


#### Check the data types of each data frame, see if any fields need to be converted

In [40]:
edgap.dtypes

NCESSCH School ID                                    int64
CT Unemployment Rate                               float64
CT Pct Adults with College Degree                  float64
CT Pct Childre In Married Couple Family            float64
CT Median Household Income                         float64
School ACT average (or equivalent if SAT score)    float64
School Pct Free and Reduced Lunch                  float64
dtype: object

In [41]:
school_info.dtypes

SCHOOL_YEAR     object
FIPST            int64
STATENAME       object
ST              object
SCH_NAME        object
                 ...  
G_AE_OFFERED    object
GSLO            object
GSHI            object
LEVEL           object
IGOFFERED       object
Length: 65, dtype: object

#### Drop unnecessary columns

#### Select columns relevant to our research question to keep

In [42]:
school_info
keep_columns = ['NCESSCH', 'MSTATE', 'MZIP', 'SCH_TYPE_TEXT', 'LEVEL']
school_info = school_info[keep_columns]

In [43]:
school_info

,NCESSCH,MSTATE,MZIP,SCH_TYPE_TEXT,LEVEL
0,1.000020e+10,AL,35220,Alternative School,High
1,1.000020e+10,AL,36057,Alternative School,High
2,1.000020e+10,AL,36057,Alternative School,High
3,1.000020e+10,AL,36057,Alternative School,High
4,1.000020e+10,AL,35206,Alternative School,High
...,...,...,...,...,...
102178,7.800030e+11,VI,802,Regular School,Elementary
102179,7.800030e+11,VI,802,Regular School,Elementary
102180,7.800030e+11,VI,802,Regular School,Elementary
102181,7.800030e+11,VI,802,Regular School,Middle


#### Id is float, convert it to 'INT64'. Also convert edgap id column from 'int64' to 'Int64' so that they are the same data type

In [44]:
school_info["NCESSCH"] = school_info["NCESSCH"].astype("Int64")
edgap["NCESSCH School ID"] = edgap["NCESSCH School ID"].astype("Int64")


<ipython-input-44-1baefd544ae1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_info["NCESSCH"] = school_info["NCESSCH"].astype("Int64")


## Convert data types, if necessary

## Are the data suitable for answering the question?

We want to perform quick exploratory data analysis to determine whether the data are sufficient to answer our question. If the data are not sufficient, we do not want to waste time doing anything that will not be productive.

## Select relevant subsets of the data

The school information data set contains a lot of information. We only need the year, school identity, location, and school type information.

Keep the columns `SCHOOL_YEAR`, `NCESSCH`, `MSTATE`, `MZIP`, `SCH_TYPE_TEXT`, `LEVEL`

## Rename columns

In [45]:
edgap = edgap.rename(columns={"NCESSCH School ID":"id",
              "CT Pct Adults with College Degree":"percent_college",
              "CT Unemployment Rate":"rate_unemployment",
              "CT Pct Childre In Married Couple Family":"percent_married",
              "CT Median Household Income":"median_income",
              "School ACT average (or equivalent if SAT score)":"average_act",
              "School Pct Free and Reduced Lunch":"percent_lunch"})
school_info = school_info.rename(columns = {"SCHOOL_YEAR":"year",
                                            "NCESSCH":"id",
                                            "MSTATE":"state",
                                            "MZIP":"zip_code",
                                            "SCH_TYPE_TEXT":"school_type",
                                            "LEVEL":"school_level"})

Rename the columns `SCHOOL_YEAR`, `NCESSCH`, `MSTATE`, `MZIP`, `SCH_TYPE_TEXT`, `LEVEL` to `year`, `id`, `state`, `zip_code`, `school_type`, `school_level`

## Join data frames

In [46]:
print(edgap)

                id  rate_unemployment  percent_college  percent_married  \
0     100001600143           0.117962         0.445283         0.346495   
1     100008000024           0.063984         0.662765         0.767619   
2     100008000225           0.056460         0.701864         0.713090   
3     100017000029           0.044739         0.692062         0.641283   
4     100018000040           0.077014         0.640060         0.834402   
...            ...                ...              ...              ...   
7981  560530200294           0.034549         0.590780         0.720077   
7982  560569000311           0.069538         0.633860         0.808841   
7983  560569500360           0.035159         0.764566         0.776570   
7984  560576200324           0.063877         0.670532         0.879324   
7985  560583000335           0.033553         0.736511         0.821128   

      median_income  average_act  percent_lunch  
0           42820.0    20.433455       0.066901  

In [47]:

merged_df = pd.merge(edgap, school_info, left_on='id', right_on='id')

print(merged_df)


                id  rate_unemployment  percent_college  percent_married  \
0     100001600143           0.117962         0.445283         0.346495   
1     100008000024           0.063984         0.662765         0.767619   
2     100008000225           0.056460         0.701864         0.713090   
3     100017000029           0.044739         0.692062         0.641283   
4     100018000040           0.077014         0.640060         0.834402   
...            ...                ...              ...              ...   
7893  560530200294           0.034549         0.590780         0.720077   
7894  560569000311           0.069538         0.633860         0.808841   
7895  560569500360           0.035159         0.764566         0.776570   
7896  560576200324           0.063877         0.670532         0.879324   
7897  560583000335           0.033553         0.736511         0.821128   

      median_income  average_act  percent_lunch state zip_code  \
0           42820.0    20.433455 

In [48]:
merged_df.dtypes

id                     Int64
rate_unemployment    float64
percent_college      float64
percent_married      float64
median_income        float64
average_act          float64
percent_lunch        float64
state                 object
zip_code              object
school_type           object
school_level          object
dtype: object

## Quality Control

Check for out-of-range values or values that do not match what we want to analyze. Either set values to `NaN` or remove the observations, as appropriate.

#### Replace all negative values in numerical columns, which are erroneous, with nan

In [49]:
columns_to_convert = ['rate_unemployment', 'percent_college', 'percent_married', 'median_income', 'average_act', 'percent_lunch']

#Delete all negative values in numerical columns
for col in columns_to_convert:
    merged_df[col] = np.where(merged_df[col] < 0, np.nan, merged_df[col])

#### Check that there are no erroneous values in categorical fields

In [50]:
print(merged_df['state'].unique())
print(merged_df['school_level'].unique())
print(merged_df['school_type'].unique())


['DE' 'FL' 'GA' 'IL' 'IN' 'KY' 'LA' 'MA' 'MI' 'MO' 'NJ' 'NY' 'NC' 'OH'
 'PA' 'TN' 'TX' 'WA' 'WI' 'WY']
['High' 'Other' 'Not reported' 'Elementary']
['Regular School' 'Alternative School' 'Special Education School'
 'Career and Technical School']


## Identify missing values

Determine whether there are missing values in the data set. Only identify them at this point; we will deal with them after creating training and testing splits of the data set.

In [51]:
null_counts = merged_df.isnull().sum()
print(null_counts)

id                    0
rate_unemployment    14
percent_college      13
percent_married      24
median_income        20
average_act           3
percent_lunch        20
state                 0
zip_code              0
school_type           0
school_level          0
dtype: int64


## Data imputation

Use an imputation method to replace missing values in the columns corresponding to predictor variables in the analysis.

#### First, Drop all rows missing an ACT score

In [52]:
merged_df.dropna(subset=['average_act'], inplace=True)

#### Then, impute missing values, using numerical fields to predict what the missing values are

In [53]:
predict_cols = ["rate_unemployment", "percent_college", "percent_married", "median_income", "percent_lunch"]

X = merged_df[predict_cols]

imputer = IterativeImputer()
df_mice = merged_df.copy()

df_mice[predict_cols] = imputer.fit_transform(X)



print(df_mice)


                id  rate_unemployment  percent_college  percent_married  \
0     100001600143           0.117962         0.445283         0.346495   
1     100008000024           0.063984         0.662765         0.767619   
2     100008000225           0.056460         0.701864         0.713090   
3     100017000029           0.044739         0.692062         0.641283   
4     100018000040           0.077014         0.640060         0.834402   
...            ...                ...              ...              ...   
7893  560530200294           0.034549         0.590780         0.720077   
7894  560569000311           0.069538         0.633860         0.808841   
7895  560569500360           0.035159         0.764566         0.776570   
7896  560576200324           0.063877         0.670532         0.879324   
7897  560583000335           0.033553         0.736511         0.821128   

      median_income  average_act  percent_lunch state zip_code  \
0           42820.0    20.433455 

#### Check that there are no na's before splitting data

In [54]:


null_counts = df_mice.isnull().sum()
print(null_counts)

id                   0
rate_unemployment    0
percent_college      0
percent_married      0
median_income        0
average_act          0
percent_lunch        0
state                0
zip_code             0
school_type          0
school_level         0
dtype: int64


#### Check that there are no erroneous min/max values

In [55]:
df_mice.describe()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
count,7895.0,7895.000000,7895.000000,7895.000000,7895.000000,7895.000000,7895.000000
mean,332381269727.175964,0.098077,0.569418,0.635273,52157.944738,20.220275,0.419235
std,132508023266.191238,0.058176,0.165746,0.194947,24163.393227,2.531748,0.237419
min,100001600143.0,0.000000,0.091493,-0.000708,659.823757,12.362637,0.000000
25%,210505501249.0,0.058581,0.451353,0.526010,36795.000000,18.650794,0.238649
50%,360008605177.0,0.085314,0.555717,0.668288,47013.000000,20.400000,0.380066
75%,422679002767.0,0.122704,0.677110,0.777502,61515.500000,21.935287,0.570574
max,560583000335.0,0.590278,1.000000,1.000000,226181.000000,32.362637,0.998729


## Train test split

We will use the test set approach to compare models predicting `average_act`.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/ML_dataset_training_validation_test_sets.png/800px-ML_dataset_training_validation_test_sets.png)

Split the data into training and testing sets. Keep 20% of the data for the test set.

In [56]:
X = df_mice.drop(columns=['average_act'])


X_train, X_test = train_test_split(df_mice, test_size=0.2, random_state=40)



print("X_train shape:", X_train.shape)

print("X_test shape:", X_test.shape)

X_train shape: (6316, 11)
X_test shape: (1579, 11)


## Create relevant derived variables as new columns

We may already know that we want to process the data to create new variables from the existing variables. However, we often start analyzing the data and realize that it is useful to create new variables derived from the existing variables. Or, we might not create any new columns. It is fine to return to modify this step after exploring the data further.

## Export the clean .csv files

Export files with the training and testing data

In [57]:
X_train.to_csv('educ_train_data.csv')
X_test.to_csv('educ_test_data.csv')
